# PyTorch-SyncBatchNorm

Example on training MNIST, code adapted from the 
[tutorial](https://github.com/pytorch/examples/tree/master/mnist)

In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn import DataParallel
from torchvision import datasets, transforms
from torch.autograd import Variable
# import the Synchronized BatchNorm here
from encoding.nn import BatchNorm2d

## Grab the MNIST dataset

In [2]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=64, shuffle=True, num_workers=1, pin_memory=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=1000, shuffle=True, num_workers=1, pin_memory=True)

## Define a convolutional neural network

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.bn1 = BatchNorm2d(10)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.max_pool2d(self.conv1(x), 2)
        x = F.relu(self.bn1(x))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

## Create DataParallel
Note that using `CUDA_VISIBLE_DEVICES` to control number of GPUs

In [4]:
model = Net()
model.cuda()
nGPUs = torch.cuda.device_count()
device_ids = [i for i in range(nGPUs)]
model = DataParallel(model, device_ids=device_ids)
# optimizer
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [5]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        target = target.cuda()
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data.item()))
            
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        target = target.cuda()
        with torch.no_grad():
            output = model(data)
            test_loss += F.nll_loss(output, target, size_average=False).data.item()
            pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.data.view_as(pred)).long().cpu().sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:
for epoch in range(1, 5):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.305259
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.134188
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.467970
Train Epoch: 1 [19200/60000 (32%)]	Loss: 1.196905
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.634555
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.618566
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.456486
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.552612
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.395239
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.564175

Test set: Average loss: 0.2674, Accuracy: 9287/10000 (93%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.349143
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.540707
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.337070
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.354382
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.309477
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.346477
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.500346
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.487035
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.497973
T